In [1]:
# Import pandas, plotly, and dash
from dash import Dash
from dash import html, dcc
from dash import callback, Output, Input
import plotly.express as px
import pandas as pd

In [17]:
# Read the excel file
df = pd.read_excel("Coffee_Shop_Sales.xlsx")

In [7]:
# Select the columns of the chart I plan to use
columns_to_select = ['transaction_id', 'transaction_date', 'transaction_qty', 'product_category', 'product_type']
filtered_df = df.loc[:, columns_to_select].copy()

In [9]:
# Convert transaction_date to a DateTime object
filtered_df['transaction_date'] = pd.to_datetime(filtered_df['transaction_date'])

In [11]:
# Filter transaction_date by year, month, and day
filtered_df['year']= filtered_df['transaction_date'].dt.year
filtered_df['month']= filtered_df['transaction_date'].dt.month
filtered_df['day']= filtered_df['transaction_date'].dt.day

In [15]:
df= pd.read_excel("Coffee_Shop_Sales.xlsx")

animatedGraph = px.histogram(
    filtered_df, 
    x="transaction_qty", 
    y="product_category", 
    animation_frame="month",
    #color="product_type",
    range_x=[0,6750],
    width = 1000,
    title = 'Total Transactions each Month by Product Type'
)

# Creating drop-down menu
app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='multiselect-category',
        options=[
            {'label': pastry, 'value': pastry} for pastry in filtered_df['product_category'].unique()
        ],
        value=filtered_df['product_category'].unique(),
        multi=True,
        placeholder='Select Categories'
        ),
    dcc.Graph(
        id='animated-graph'
    )
])

# Call the drop-down menu that was created
@app.callback(
    Output('animated-graph', 'figure'),
    Input('multiselect-category','value')
)

# Create an Animated Histogram
def update_graph(categories): 
    filtered_data = filtered_df[filtered_df['product_category'].isin(categories)]
    
    animatedGraph = px.histogram(
        filtered_data, 
        x="transaction_qty", 
        y="product_category", 
        animation_frame="month",
        color="product_type",
        range_x=[0,14000],
        width = 1000,
        title = 'Total Transactions Made Each Month by Product Category'
        )

# Update the x and y axis title, resize, and add subtitle
    animatedGraph.update_layout(
        xaxis_title = "Transaction Quantity Count",
        yaxis_title = "Product Category",
        title_x = 0.5,
        title_subtitle_text = 'This histogram shows coffee shop sales from January to June. Items sold are displayed in 9 categories that can be filtered <br> at the top of the chart. Each category displays the transaction quantity count. The graph is color-coded by product types.',
        font_size = 12,
        legend_title_text='Product Type',
        plot_bgcolor = 'white'             
    )
    return animatedGraph

app.run(debug=True)